In [1]:
import hopsworks
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import xgboost 

In [2]:
import hsfs

# 1. Login
project = hopsworks.login()

# 2. Get the Feature Store (This triggers the metadata check)
try:
    fs = project.get_feature_store("A1ID2223")
    print(f"Successfully connected to Feature Store: {fs.name}")
except Exception as e:
    print(f"Feature Store Connection Error: {e}")

# 3. Check versions
print(f"HSFS Version: {hsfs.__version__}")

2025-12-30 20:49:43,248 INFO: Initializing external client
2025-12-30 20:49:43,249 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-12-30 20:49:44,905 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1267871
Successfully connected to Feature Store: a1id2223_featurestore
HSFS Version: 4.2.10


In [3]:
sentiment_feature_group = fs.get_feature_group(name="sentiments")
opening_prices_feature_group = fs.get_feature_group(name="opening_prices", version=2)

# Join without filter first to check data
sentiment_opening_price_view = (
    sentiment_feature_group.select_all()
    .join(opening_prices_feature_group.select_all(), on=["date"])
)

df = sentiment_opening_price_view.read()

# Rename columns to remove suffixes
df = df.rename(columns={
    'opening_prices_open': 'open',
    'opening_prices_target_open': 'target_open'
})

# Filter out rows with null target_open after read
df = df[df['target_open'].notna()]

print(f"Rows after filtering: {len(df)}")

2025-12-30 20:49:46,357 WARNING: VersionWarning: No version provided for getting feature group `sentiments`, defaulting to `1`.

2025-12-30 20:49:47,150 WARNING: Ambiguous features detected during query construction.The feature `date` is present in feature groups ['opening_prices version 2', 'sentiments version 1']. Automatically prefixing features selected using these feature groups with the feature group name.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.23s) 
Rows after filtering: 1135


In [4]:
print(df.sort_values("date").head())
df.info()

                          date  sentiment_polarity  sentiment_neg  \
903  2016-02-19 00:00:00+00:00               0.994          0.023   
1387 2017-10-05 00:00:00+00:00               0.997          0.008   
1201 2017-11-27 00:00:00+00:00               0.997          0.008   
805  2017-11-30 00:00:00+00:00               0.989          0.021   
1297 2018-01-31 00:00:00+00:00               0.995          0.009   

      sentiment_neu  sentiment_pos       open  target_open  
903           0.869          0.108  21.762466    21.832740  
1387          0.925          0.067  35.978361    36.162716  
1201          0.926          0.065  40.995288    40.819656  
805           0.804          0.174  39.913317    39.800894  
1297          0.937          0.054  39.079592    39.149856  
<class 'pandas.core.frame.DataFrame'>
Index: 1135 entries, 0 to 1544
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----       

In [5]:
feature_view = fs.get_or_create_feature_view(
    name="sentiment_and_opening_price_view",
    version=1,
    query=sentiment_opening_price_view,
    description="sentiment + opening price view",
    labels=['target_open']
)

In [6]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start="2024-10-01"
)

# Drop date column - it's for temporal ordering, not a feature
X_train = X_train.drop(columns=['date'], errors='ignore')
X_test = X_test.drop(columns=['date'], errors='ignore')

# Remove rows with NaN targets (last day from backfill, today from daily pipeline)
# Fix: squeeze() or use .values.ravel() to get 1D array for proper boolean indexing
train_mask = y_train.notna().squeeze()
X_train = X_train[train_mask]
y_train = y_train[train_mask]

test_mask = y_test.notna().squeeze()
X_test = X_test[test_mask]
y_test = y_test[test_mask]

print(f"Filtered train: {len(X_train)} rows, test: {len(X_test)} rows")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 
2025-12-30 20:49:58,420 WARNING: VersionWarning: Incremented version to `9`.

Filtered train: 1093 rows, test: 42 rows


In [7]:
# Debug: Check for NaN values
print(f"y_train type: {type(y_train)}")
print(f"y_train shape: {y_train.shape}")
print(f"NaN count in y_train: {y_train.isna().sum()}")
print(f"\ny_train head:")
print(y_train.head())

y_train type: <class 'pandas.core.frame.DataFrame'>
y_train shape: (1093, 1)
NaN count in y_train: opening_prices_target_open    0
dtype: int64

y_train head:
   opening_prices_target_open
0                   52.729775
1                   41.576751
2                   48.915207
3                   45.511297
4                   44.696488


In [8]:
# Validate train/test split
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(f"\nFeatures: {X_train.columns.tolist()}")
print(f"\nTrain date range: {X_train.index.min()} to {X_train.index.max()}")
print(f"Test date range: {X_test.index.min()} to {X_test.index.max()}")
print(f"\nTarget stats - Train mean: {float(y_train.mean()):.2f}, Test mean: {float(y_test.mean()):.2f}")

Train shape: (1093, 5), Test shape: (42, 5)

Features: ['sentiment_polarity', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos', 'opening_prices_open']

Train date range: 0 to 1538
Test date range: 169 to 1544
2025-12-30 20:49:58,440 WARNING: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead


Target stats - Train mean: 151.45, Test mean: 229.57


In [9]:
xgb_regressor = xgboost.XGBRegressor(random_state=42)
xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [10]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
import numpy as np

y_pred = xgb_regressor.predict(X_test)
mse = mean_squared_error(y_test.iloc[:,0], y_pred)
r2 = r2_score(y_test.iloc[:,0], y_pred)
mae = mean_absolute_error(y_test.iloc[:,0], y_pred)
rmse = np.sqrt(mse)

print(f"MSE:  {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R²:   {r2:.4f}")

2025-12-30 20:49:58,521 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2025-12-30 20:49:58,521 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2025-12-30 20:49:58,522 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2025-12-30 20:49:58,522 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2025-12-30 20:49:58,523 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2025-12-30 20:49:58,524 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

In [11]:
import os

model_dir = f"sentiment_stock_price_model_AAPL"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [12]:
xgb_regressor.save_model(model_dir + "/model.json")
print(f"Model saved to {model_dir}")

Model saved to sentiment_stock_price_model_AAPL


In [13]:
res_dict = { 
    "MSE": float(mse),
    "RMSE": float(rmse),
    "MAE": float(mae),
    "R2": float(r2),
}

In [14]:
mr = project.get_model_registry()

model_name = model_dir

aq_model = mr.python.create_model(
    name=model_name, 
    metrics=res_dict,
    feature_view=feature_view,
    description=f"Opening price predictor for AAPL stock based on sentiment",
)

aq_model.save(model_dir)
print(f"Model '{model_name}' uploaded to registry")

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/sambarati/Documents/GitHub/nlp-stock-prediction/notebooks/sentiment_stock_price_model_AAPL/mo…

Uploading /Users/sambarati/Documents/GitHub/nlp-stock-prediction/notebooks/model_schema.json: 0.000%|         …

Model created, explore it at https://c.app.hopsworks.ai:443/p/1267871/models/sentiment_stock_price_model_AAPL/2
Model 'sentiment_stock_price_model_AAPL' uploaded to registry


In [15]:
# Check the actual schema that was saved
print("Input schema saved to model registry:")
print(f"Features used for training: {X_train.columns.tolist()}")
print(f"\nFeature count: {len(X_train.columns)}")
print("\nNote: The schema includes 'date' but it was dropped during training!")
print("This may cause issues during inference.")

Input schema saved to model registry:
Features used for training: ['sentiment_polarity', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos', 'opening_prices_open']

Feature count: 5

Note: The schema includes 'date' but it was dropped during training!
This may cause issues during inference.
